# 🧪 Lab 9: Working with Image Data for Agricultural Analysis

## 🎯 Objectives

In this lab, you will:

- Understand how image data is structured and represented in Python
- Learn to load and visualize images using popular Python libraries
- Explore individual RGB color channels and grayscale conversions
- Perform basic image processing techniques such as masking and thresholding
- Apply pixel-based analysis to quantify features like canopy coverage
- Interpret image statistics to extract useful agricultural information

---

## 🌱 Introduction

In modern agriculture, images from cameras, drones, and satellites provide vital data for monitoring crops, detecting stress, and assessing field conditions. These images are essentially arrays of pixel values—numbers that can be processed, visualized, and analyzed using Python.

By learning how to manipulate image data, you can develop tools to:

- Estimate plant health
- Track canopy development
- Detect early signs of disease
- Quantify soil exposure, greenness, and more

In this lab, we will begin with simple image handling and gradually move toward meaningful pixel-level analysis to support precision agriculture.


## 🖼️ Reading and Displaying Images

### 📌 What is an Image?

In Python, an image is typically represented as a 2D or 3D array:

- **Grayscale Image**: 2D array (Height × Width), where each value is pixel brightness (0–255)
- **RGB Image**: 3D array (Height × Width × 3), with values for Red, Green, and Blue channels

These numerical representations allow you to perform image analysis using standard Python tools like NumPy and Matplotlib.

---

### 📥 Loading Images in Python

You can use either `matplotlib.pyplot`, `PIL` (Pillow), or `cv2` (OpenCV) to load images. For simplicity, we’ll use `matplotlib.pyplot.imread()`.



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import requests
from PIL import Image
from io import BytesIO

# Load image from URL
url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/3/36/Plant_green_leaves.jpg/640px-Plant_green_leaves.jpg'
response = requests.get(url)
img = Image.open(BytesIO(response.content))

# Convert to NumPy array
img_np = np.array(img)

# Display image
plt.imshow(img_np)
plt.axis('off')
plt.title('Online Plant Image')
plt.show()

# Print image shape and data type
print("Image shape:", img_np.shape)
print("Pixel data type:", img_np.dtype)


## 🎨 RGB Channel Analysis and Grayscale Conversion

Color images in the RGB format contain three color channels:

- **Red**: Highlights red-toned pixels
- **Green**: Highlights vegetation and healthy plant tissue
- **Blue**: Often used in background or water detection

These three channels are stacked into a 3D array: `(Height, Width, 3)`. You can isolate each channel for analysis or visualization.

---

### 🔄 Converting RGB to Grayscale

Grayscale images combine the color channels into a single intensity channel, removing color but preserving brightness. A standard method to convert to grayscale is:

**Gray = 0.2989 × R + 0.5870 × G + 0.1140 × B**

This weighted average reflects human brightness perception.

Grayscale conversion is useful for:

- Histogram analysis
- Thresholding and masking
- Edge or shape detection


In [ ]:
# Extract RGB channels
red_channel = img_np[:, :, 0]
green_channel = img_np[:, :, 1]
blue_channel = img_np[:, :, 2]

# Plot RGB channels
fig, axs = plt.subplots(1, 3, figsize=(15, 4))
axs[0].imshow(red_channel, cmap='Reds')
axs[0].set_title('Red Channel')
axs[0].axis('off')

axs[1].imshow(green_channel, cmap='Greens')
axs[1].set_title('Green Channel')
axs[1].axis('off')

axs[2].imshow(blue_channel, cmap='Blues')
axs[2].set_title('Blue Channel')
axs[2].axis('off')

plt.tight_layout()
plt.show()

# Convert to grayscale
gray_img = 0.2989 * red_channel + 0.5870 * green_channel + 0.1140 * blue_channel

# Show grayscale image
plt.figure(figsize=(5, 5))
plt.imshow(gray_img, cmap='gray')
plt.title('Grayscale Image')
plt.axis('off')
plt.show()


## 🔧 Basic Image Processing for Agricultural Analysis

Once image data is loaded and explored, we can apply simple transformations to highlight specific features—especially vegetation.

---

### 🌿 Detecting Vegetation with a "Greenness" Index

Healthy plants typically reflect more green light than red or blue. One way to highlight vegetation is to use a **greenness index**, such as:

**Greenness Index = (G − R) / (G + R)**

- Higher values → more green, likely vegetation
- This approach is similar to NDVI, but for RGB images

---

### 🎯 Thresholding and Binary Masks

We can apply a **threshold** to the greenness index to isolate areas with strong vegetation signals. This produces a binary mask:

- `1` (or `True`) = vegetation detected
- `0` (or `False`) = background

This allows us to:
- Visualize vegetation clearly
- Quantify canopy coverage (e.g., % of green pixels)


In [1]:
# Convert pixel values to float (in case they're uint8)
red = red_channel.astype(float)
green = green_channel.astype(float)

# Calculate "greenness index" (similar to NDVI but with RGB)
greenness_index = (green - red) / (green + red + 1e-5)  # avoid division by zero

# Show greenness index
plt.figure(figsize=(6, 5))
plt.imshow(greenness_index, cmap='YlGn')
plt.colorbar(label='Greenness Index')
plt.title('Greenness Index (G-R)/(G+R)')
plt.axis('off')
plt.show()

# Apply threshold to highlight vegetation
threshold = 0.1
veg_mask = greenness_index > threshold

# Plot binary vegetation mask
plt.figure(figsize=(6, 5))
plt.imshow(veg_mask, cmap='gray')
plt.title('Vegetation Mask')
plt.axis('off')
plt.show()

# Calculate % of green pixels
green_percent = np.sum(veg_mask) / veg_mask.size * 100
print(f"Estimated canopy coverage: {green_percent:.2f}% of image area")


NameError: name 'red_channel' is not defined

## 📊 Image Histograms and Pixel Statistics

Analyzing the distribution of pixel values helps quantify characteristics such as brightness, contrast, and greenness.

### 📈 Histograms
A **histogram** shows how pixel values are distributed across an image.

- For grayscale: histogram shows brightness levels
- For RGB: each channel has its own histogram (Red, Green, Blue)

### 📉 Summary Statistics
You can compute useful descriptive statistics:
- **Mean** and **Median**: average brightness or color intensity
- **Standard Deviation**: contrast or variability
- **Min/Max**: extreme pixel values (e.g., shadows or overexposure)

This helps in:
- Monitoring image quality
- Comparing plant color distributions
- Assessing overall lighting and exposure


In [ ]:
# Compute and display histogram for grayscale image
plt.figure(figsize=(6, 4))
plt.hist(gray_img.ravel(), bins=256, color='gray', alpha=0.8)
plt.title('Grayscale Histogram')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# Show RGB histograms
plt.figure(figsize=(8, 5))
colors = ['red', 'green', 'blue']
channels = [red_channel, green_channel, blue_channel]

for color, channel in zip(colors, channels):
    plt.hist(channel.ravel(), bins=256, color=color, alpha=0.5, label=f'{color.capitalize()} Channel')

plt.title('RGB Histograms')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True)
plt.show()

# Print basic statistics for green channel
print("Green Channel Statistics:")
print(f"Mean: {np.mean(green_channel):.2f}")
print(f"Median: {np.median(green_channel):.2f}")
print(f"Standard Deviation: {np.std(green_channel):.2f}")
print(f"Min: {np.min(green_channel)}, Max: {np.max(green_channel)}")


## 📝 Exercises: Image Analysis for Plant Monitoring

Complete the following exercises to apply what you've learned in this lab.

---

### 🌿 Part 1: Load and Inspect an Online Image

1. Load an RGB image from an online URL using `requests` and `PIL`.
2. Display the image using `matplotlib.pyplot.imshow`.
3. Print the shape and data type of the image array.

---

### 🎨 Part 2: Analyze RGB Channels

1. Separate the Red, Green, and Blue channels.
2. Visualize each channel using `imshow` with appropriate colormaps.
3. Compute and display the histograms for each channel.

---

### ⚪ Part 3: Grayscale Conversion and Analysis

1. Convert the RGB image to grayscale using the weighted method:

   $$
   \text{Gray} = 0.2989 \times R + 0.5870 \times G + 0.1140 \times B
   $$

2. Display the grayscale image.
3. Plot a histogram of grayscale values.
4. Report the mean and standard deviation of the grayscale pixel values.

---

### 🌱 Part 4: Vegetation Detection Using a Greenness Index

1. Compute a greenness index using:

   $$
   \text{Greenness Index} = \frac{G - R}{G + R}
   $$

2. Visualize the index using a continuous colormap (e.g., `YlGn`).
3. Create a binary vegetation mask by applying a threshold (e.g., 0.1).
4. Visualize the mask.
5. Calculate the percentage of the image identified as vegetation.

---

### 💡 Bonus Challenge (Optional)

Use two different plant images (e.g., early vs. late stage growth) and compare:
- Histogram distributions
- Vegetation coverage percentage

Reflect on how pixel-level analysis helps monitor plant development in agriculture.
